In [1]:
import numpy as np
import vector
import scipy 

In [2]:
# Useful constants:

mu0 = 4*np.pi*1e-7   # H/m          Permeability of free space
eps0 = 8.854*1e-12   # F/m          Permitivity of free space



In [3]:
# Derived constants:

c = 1/np.sqrt(mu0*eps0)          # m/s      Velocity of propagation in free space
eta0 = np.sqrt(mu0/eps0)         # Ohm      Intrinsic impedance of free space


In [4]:
def linPolEField(E0Inc, thetaInRad, phiInRad, alphaInRad):

    # TODO:
    # Add checks on the angle values: 0<=thetaInRad<=np.pi ; 0<=phiInRad<2*np.pi; 0<=alphaInRad<2*np.pi 
    # (alpha can also be defined negative, as long as within similar limits)
    
    if((thetaInRad==0)|(thetaInRad==np.pi)):
        EIncVector = vector.obj(x = E0Inc*np.cos(phiInRad), y = E0Inc*np.sin(phiInRad), z = 0.0)
    else:         
        # Projection of EIncVector on alpha plane (i.e. plane containing EIncVector)
        Eu_i = E0Inc*np.cos(alphaInRad)
        Ev_i = E0Inc*np.sin(alphaInRad)
        # Projection of Eu_i and Ev_i on xy-plane and z:
        Euxy_i = Eu_i*np.cos(thetaInRad)
        Euz_i = Eu_i*np.sin(thetaInRad)
        Evxy_i = Ev_i  #  Ev_i is invariant with thetaIn (Ev_i is perpendicular to pz-plane by construction)
        Evz_i = 0.0   #  Ev_i is perpendicular to pz-plane by construction (i.e. this is no magic-number)
        # Projection of Euxy_i, Euz_i, Evxy_i and Evz_i on xyz system:
        EzProj = Euz_i + Evz_i
        ExProj = (-1.0*Euxy_i*np.cos(phiInRad)  + (-1.0*Evxy_i*np.sin(phiInRad)))   
        EyProj = (-1.0*Euxy_i*np.sin(phiInRad)  + Evxy_i*np.cos(phiInRad))
        
        EIncVector = vector.obj(x = ExProj, y = EyProj, z = EzProj)

    print("Ex: ",EIncVector.x)
    print("Ey: ",EIncVector.y)
    print("Ez: ",EIncVector.z)

    return EIncVector

class planeWave:
    """The planeWave class contains the definition of a plane wave propagating in space.
        The plane wave is defined here in the frequency domain (i.e. omega), and a harmonic
        field is assumed.
        The object is defined in terms of a vector object 'E0Vector' ('Vector' library) and 
        another vector object 'waveVector'."""
    
    def __init__(self, E0Vector, waveVector):
        self.E0Vector = E0Vector
        self.waveVector = waveVector
    def __str__(self):
        return f"{self.E0Vector}*(exp(i*dot({self.waveVector},position)))"   
    
    #Methods of class planeWave:
    
    def obsEField(self, obs):  # Compute E field at location obs
        """'self.obsEField(obs)' which returns the electric field (including the phase term, therefore 
        complex) of the plane wave at location 'obs'. """
        return self.E0Vector*np.exp(1j*self.waveVector.dot(obs))

In [5]:
# TL parameters:

R = 1       # Ohm/m
L = 1e-6    # H/m
C = 1e-12   # F/m
G = 1       # 1/Ohm / m

**Impingin plane wave**:
The impinging plane wave is defined as follows:
![impingingPlaneWaveDef](images/impingingPlaneWaveDef.JPG) 

In [6]:
# Impinging plane wave:

E0Inc = 1        # RMS value of the field intensity, V/m
f = 1e9          # Hz

#Incidence:
# Note for user:
# If thetaIn = 0 or 180, the polarization of E is associated simply to the azimuthal angle phiIn 
# (alphaIn is not used, cause is undefined)
thetaIn = 180      # Degrees, 
phiIn = 135        # Degrees
alphaIn = -30      # Degrees



In [7]:
# Impinging field derived quantities

thetaInRad = thetaIn*np.pi/180     # 0 < thetaIn< 180
phiInRad = phiIn*np.pi/180
alphaInRad = alphaIn*np.pi/180


omega_ = 2*np.pi*f    # rad/s
lambda_ = c/f         # m
wavenumber = omega_*np.sqrt(mu0*eps0)

# Ingoing wave (therefore directed towards -1.0*rho):
waveVector = vector.obj(phi = phiInRad, theta = thetaInRad, rho = -1*wavenumber*np.sin(thetaInRad)) 

# Assemble EIncVector from linearly polarized planewave definition:
EIncVector = linPolEField(E0Inc, thetaInRad, phiInRad, alphaInRad)  

# Create planeWave instance:
incPw = planeWave(EIncVector, waveVector) 

Ex:  -0.7071067811865475
Ey:  0.7071067811865476
Ez:  0.0


In [9]:
# Test obsEfield method:
# Compute Electric field at obs. point xyz
obs = vector.obj(x = 0, y = 0, z = 0)
incPw.obsEField(obs)

VectorObject3D(x=(-0.7071067811865475+0j), y=(0.7071067811865476+0j), z=0j)

In [ ]:
# TL derived quantities:

Z = R + 1j*omega_*L
Y = G + 1j*omega_*C

gamma_ = np.sqrt(Z*Y)
Z0 = np.sqrt(Z/Y)


**Note from Vector library and coordinates:**

For 3D vectors (e.g. $(E_x, E_y, E_z)$), the angles in [spherical coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system) are defined as follows:

- $\varphi$ (aka azimuthal), angle between the projection of *vector* on the $xy$ plane and the $x$ axis: **phi**
- $\theta$ (aka polar), angle between *vector* and $z$ axis: **theta**
- $r$ (aka *radial distance*), distance to origin: implemented via **rho** ($\rho$ = $r$*$sin(\theta)$)